### AI Agent from Scratch on Tenstorrent

In [1]:
# SPDX-FileCopyrightText: (c) 2025 Tenstorrent AI ULC
#
# SPDX-License-Identifier: Apache-2.0
import json
import os

from openai import OpenAI
import requests

### Custom function to fetch current weather conditions 

In [2]:
def fetch_weather(location):
    params = {
        "q": location,
        "key": os.environ["WEATHER_API_KEY"]
    }
    base_url = "http://api.weatherapi.com/v1/current.json"

    res = requests.get(base_url, params=params)
    data = res.json()
    current_data = data['current']

    temp_string = f"{current_data['temp_f']} °F"
    
    condition_string = current_data['condition']['text']
    location_string = f"{data['location']['name']}, {data['location']['region']}"

    return f"Weather in {location_string}: {temp_string}, {condition_string}.\n"

### Set Tenstorrent endpoint URL and create client

In [3]:
tt_base_url = os.environ["TT_BASE_URL"]  # "https://<...>.koyeb.app/v1"

client = OpenAI(base_url=tt_base_url, api_key="null")
models = client.models.list()
model_id = models.data[0].id

## Overview
1. User submits query to LLM
2. LLM determines if a tool call is necessary.  If so, it will formulate a function call with parameters
    - i.e. If the query is `"What's the weather in Chicago?"` the LLM should return a tool call of `{'location': 'Chicago'}`
3. We use the JSON function call to run the actual `fetch_weather` function with the LLM's provided arguments.
4. The tool call and context is passed back to the LLM to generate a final response.

### 1. Define tools in OpenAI function calling schema

In [4]:
tools = [{
    "type": "function",
    "function": {
        "name": "fetch_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string", 
                    "description": "The city, region, or place, like 'Austin', 'New York City', or 'Barcelona'"
                }
            },
            "required": ["location"],
            "additionalProperties": False
        }
    }
}]

### 2. Create user prompt and system message
The system message will instruct the LLM on how to act and respond.

In [5]:
user_message = {
    "role": "user", 
    "content": "I'm going to Anchorage, how should I dress?"
}

system_message = {
    "role": "system",
    "content":
        """
        You are a helpful assistant that gives practical outfit recommendations based on the current weather conditions provided.\n
        When given details like temperature, rain, wind, and other weather factors, suggest appropriate clothing items and accessories for comfort.\n
        Keep your suggestions concise, relevant, and user-friendly.\n
        Example:
        - "It's cold and windy, wear a warm coat and a scarf."
        - "Expect rain, bring an umbrella and wear waterproof shoes."
        You must repeat the weather data so the user knows the exact forecast.
        """
}

### 3. Call completion to get tool call parameters

In [8]:
tool_call_response = client.chat.completions.create(
    model=model_id,
    messages=[system_message, user_message],
    tools=tools,
    tool_choice="auto"
)

tool_call = tool_call_response.choices[0].message.tool_calls[0]

parameters = json.loads(tool_call.function.arguments)

print(f"[TOOL CALL] Parameters: {parameters}")

[TOOL CALL] Parameters: {'location': 'Anchorage'}


### 4. Run function with tool call parameters

In [9]:
context = fetch_weather(**parameters)

print(f"[TOOL CALL] Output: {context}")

[TOOL CALL] Output: Weather in Anchorage, Alaska: 63.0 °F, Partly Cloudy.



### 5. Give tool call and tool output back to model as context

In [10]:
assistant_message = {
    "role": "assistant",
    "tool_calls": [{
        "id": tool_call.id,
        "type": "function",
        "function": {
            "name": tool_call.function.name,
            "arguments": tool_call.function.arguments
        }
    }]
}

tool_call_message = {
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": context
}

messages = [system_message, user_message, assistant_message, tool_call_message]

for message in messages:
    print(message)

{'role': 'system', 'content': '\n        You are a helpful assistant that gives practical outfit recommendations based on the current weather conditions provided.\n\n        When given details like temperature, rain, wind, and other weather factors, suggest appropriate clothing items and accessories for comfort.\n\n        Keep your suggestions concise, relevant, and user-friendly.\n\n        Example:\n        - "It\'s cold and windy, wear a warm coat and a scarf."\n        - "Expect rain, bring an umbrella and wear waterproof shoes."\n        You must repeat the weather data so the user knows the exact forecast.\n        '}
{'role': 'user', 'content': "I'm going to Anchorage, how should I dress?"}
{'role': 'assistant', 'tool_calls': [{'id': 'chatcmpl-tool-4bbfc99e98c0434a92c66a8129c7e04c', 'type': 'function', 'function': {'name': 'fetch_weather', 'arguments': '{"location": "Anchorage"}'}}]}
{'role': 'tool', 'tool_call_id': 'chatcmpl-tool-4bbfc99e98c0434a92c66a8129c7e04c', 'content': '

### 6. Run final chat completion for final response.

In [12]:
stream = client.chat.completions.create(
    model=model_id,
    messages=messages,
    stream=True
)

for chunk in stream:
    delta = chunk.choices[0].delta.content or ""
    print(delta, end="", flush=True)

It's 63°F in Anchorage, Alaska, with partly cloudy conditions. To dress comfortably, I suggest wearing:

- A light sweater or a fleece jacket for layering.
- Comfortable jeans or trousers.
- A pair of sturdy shoes or boots suitable for walking.
- Consider bringing a lightweight rain jacket or poncho, as occasional rain showers are possible.

Please check the latest forecast before your trip to ensure the most accurate information.